# Code for analysis of GM to WM contrast in SC images and SC QSM processing steps

In [3]:
import nibabel as nib
import numpy as np  
import sys
import os
import pandas as pd

In [4]:
from scipy.stats import ttest_ind
from scipy.stats import mannwhitneyu
from scipy import stats

In [5]:
import importlib
from algo_comp_from_folder import calculate_t_statistic_between_gm_wm

In [4]:
# Lets begin analyzing our best looking magnitude
hc2_m1_mag_img = nib.load(r"E:\msc_data\sc_qsm\swiss_data_mk2\QSM_processing\hc2\m1\hc2_m1_mag.nii.gz")
hc2_m1_mag_data = hc2_m1_mag_img.get_fdata()
# Now lets load the custom masks
hc2_m1_gm_msk_data = nib.load(r"E:\msc_data\sc_qsm\swiss_data_mk2\QSM_processing\hc2\m1/custom_hc2_m1_gm_msk.nii.gz").get_fdata()
hc2_m1_wm_msk_data = nib.load(r"E:\msc_data\sc_qsm\swiss_data_mk2\QSM_processing\hc2\m1\custom_hc2_m1_wm_msk.nii.gz").get_fdata()

### Lets calculate SNR of one of the in-vivo meGRE images

In [8]:
import os, sys
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)
print(parent_dir)

c:\Users\Admin\Documents\msc_project\Image-processing-strategies


In [10]:
import monkey_tools.snr_calc
importlib.reload(monkey_tools.snr_calc)

from monkey_tools.snr_calc import snr_calc  # re-import updated function

In [ ]:
# Calculate SNR through echoes
hc2_m1_mag_path = r"E:\msc_data\sc_qsm\swiss_data_mk2\QSM_processing\hc2\m1\hc2_m1_mag.nii.gz"
signal_mask_path = r"E:\msc_data\sc_qsm\swiss_data_mk2\QSM_processing\hc2\m1\custom1_sc_msk.nii.gz" # Using the custom SC mask
noise_mask_path = r"E:\msc_data\sc_qsm\swiss_data_mk2\QSM_processing\hc2\m1\noisy_air_mask.nii.gz"
snr_hc2_m1, sigma_this_acq = snr_calc(hc2_m1_mag_path, signal_mask_path, noise_mask_path)
print("SNR for HC2 M1 on using a spinal cord mask is: ", snr_hc2_m1)
print("Sigma for this acq: ", sigma_this_acq)

SNR for HC2 M1 on using a spinal cord mask is:  [66.93923521 62.37709702 57.90741596 53.20093426 48.83005813]
Sigma for this acq:  1.3281895278320242


In [13]:
# Calculate SNR through echoes
sevenT_m1_mag_path = r"E:\msc_data\sc_qsm\swiss_data_7T\QSM_processing\m1\fixed_NS_V_S17_mag.nii.gz"
sevenT_m1_sc_mask_path = r"E:\msc_data\sc_qsm\swiss_data_7T\QSM_processing\m1\swiss_7T_mk1_m1_sc_msk.nii.gz" 
sevenT_m1_noise_mask_path = r"E:\msc_data\sc_qsm\swiss_data_7T\QSM_processing\m1\swiss_7T_mk1_m1_noise_msk.nii.gz"
snr_sevenT_m1, sigma_sevenT_m1 = snr_calc(sevenT_m1_mag_path, sevenT_m1_sc_mask_path, sevenT_m1_noise_mask_path)
print("SNR for Swiss 7T data, signal from the spinal cord mask is: ", snr_sevenT_m1)
print("Sigma for this acq: ", sigma_sevenT_m1)

SNR for Swiss 7T data, signal from the spinal cord mask is:  [10.33865028  8.77999293  7.16613821  5.84783546  4.93598338]
Sigma for this acq:  62.773582417009955


In [56]:
a = nib.load(noise_mask_path)
a.shape

(384, 384, 16, 5)

In [52]:
np.mean(snr_hc2_m1)

np.float64(57.850948114182174)

In [ ]:
# Now, lets calculate the SNR that we expect to have at 1ms, to use that SNR target for our simulations


In [ ]:
print(f"WM Mean: {wm_mean:.4f}, GM Mean: {gm_mean:.4f}, Mean Difference (GM - WM): {mean_diff:.4f}")
print("WM voxel count:", wm_vals.size)
print("GM voxel count:", gm_vals.size)
print("WM variance:", np.var(wm_vals))
print("GM variance:", np.var(gm_vals))

WM Mean: 0.0106, GM Mean: -0.0021, Mean Difference (GM - WM): -0.0128
WM voxel count: 477
GM voxel count: 2252
WM variance: 0.00014923495286389455
GM variance: 0.00011575811589836249


In [ ]:
# Same SNR calculation for more datasets:


# Continue to in-vivo SC QSM analysis

In [7]:
hc2_m1_opt_pdf_opt_tkd_data = nib.load(r"E:\msc_data\sc_qsm\swiss_data_mk2\QSM_processing\hc2\m1\chi_map\pdf_to_tkd\opt_to_opt/Sepia_Chimap.nii.gz").get_fdata()

In [5]:
nSlices = hc2_m1_mag_img.shape[2]
nSlices

16

In [6]:
contrast_values = np.full(nSlices, np.nan)

for s in range(nSlices):
    wm_slice = hc2_m1_wm_msk_data[:, :, s].astype(bool)
    gm_slice = hc2_m1_gm_msk_data[:, :, s].astype(bool)

    wm_vals = hc2_m1_mag_data[:, :, s][wm_slice]
    gm_vals = hc2_m1_mag_data[:, :, s][gm_slice]

    if wm_vals.size > 0 and gm_vals.size > 0:
        mu_wm = wm_vals.mean()
        mu_gm = gm_vals.mean()

        contrast_values[s] = (mu_gm - mu_wm) / (mu_gm + mu_wm)  # signed contrast

# contrast_values now contains the GM–WM contrast for each slice
print(contrast_values)


[       nan        nan 0.1012321  0.03194563 0.05406769 0.08772266
 0.06536509 0.0661796  0.07663165 0.07719194 0.06496995 0.08389183
 0.06915417        nan        nan        nan]


# <span style="color:orange"> Significant difference of mean in Chimaps ?// *For SC QSM* </br>
In this section we'll perform a statistical test looking to see if the difference between of the GM and WM mean is significant or not.


In [8]:
# Load the chimap you want to analyuze
# chimap_img = nib.load()
chimap_data = hc2_m1_opt_pdf_opt_tkd_data

hc1m1_chimap_path = r"E:\msc_data\sc_qsm\swiss_data_mk2\QSM_processing\hc1\m1\chi_map\pdf_to_tkd/opt_to_opt\Sepia_Chimap.nii.gz"
wm_hc1m1_mask_path =r"E:\msc_data\sc_qsm\swiss_data_mk2\QSM_processing\hc1\m1\custom_hc1_m1_gm_msk.nii.gz"
gm_hc1m1_mask_path = r"E:\msc_data\sc_qsm\swiss_data_mk2\QSM_processing\hc1\m1\custom_hc1_m1_wm_msk.nii.gz"

# Now load the masks or custom masks you want to use to calculate the mean 
wm_mask_data = nib.load(r"E:\msc_data\sc_qsm\swiss_data_mk2\QSM_processing\hc2\m1\custom_hc2_m1_gm_msk.nii.gz").get_fdata()
gm_mask_data = nib.load(r"E:\msc_data\sc_qsm\swiss_data_mk2\QSM_processing\hc2\m1\custom_hc2_m1_wm_msk.nii.gz").get_fdata()
                      

In [9]:
t_stat_hc1m1, p_val_hc1m1 = calculate_t_statistic_between_gm_wm(hc1m1_chimap_path, wm_hc1m1_mask_path, gm_hc1m1_mask_path)

T-statistic: -14.493824, p-value: 8.690809e-41


In [23]:
# Extract voxel values
wm_vals = chimap_data[wm_mask_data == 1]
gm_vals = chimap_data[gm_mask_data == 1]

# Remove NaN/inf values
wm_vals = wm_vals[np.isfinite(wm_vals)]
gm_vals = gm_vals[np.isfinite(gm_vals)]

# Means
wm_mean = np.mean(wm_vals)
gm_mean = np.mean(gm_vals)
mean_diff = gm_mean - wm_mean  # Substract WM because it should always be negative

In [30]:
print(f"WM Mean: {wm_mean:.4f}, GM Mean: {gm_mean:.4f}, Mean Difference (GM - WM): {mean_diff:.4f}")
print("WM voxel count:", wm_vals.size)
print("GM voxel count:", gm_vals.size)
print("WM variance:", np.var(wm_vals))
print("GM variance:", np.var(gm_vals))

WM Mean: 0.0106, GM Mean: -0.0021, Mean Difference (GM - WM): -0.0128
WM voxel count: 477
GM voxel count: 2252
WM variance: 0.00014923495286389455
GM variance: 0.00011575811589836249


Not a big difference in voxel counts indicates that there may not be big imbalance (and if there is, this may not be why)

Nonzero viarance is good, also variance is similar which indicates that independendent t-test is a good fit :)

In [27]:
# Check no Nans, because this messes up the stats
print("NaNs in WM:", np.isnan(wm_vals).sum())
print("NaNs in GM:", np.isnan(gm_vals).sum())
print("Infs in WM:", np.isinf(wm_vals).sum())
print("Infs in GM:", np.isinf(gm_vals).sum())


NaNs in WM: 0
NaNs in GM: 0
Infs in WM: 0
Infs in GM: 0


In [15]:
# Just in case remove NaN/inf values again
wm_vals_clean = wm_vals[np.isfinite(wm_vals)]
gm_vals_clean = gm_vals[np.isfinite(gm_vals)]

In [28]:
t_stat, p_val = ttest_ind(wm_vals, gm_vals, equal_var=False)
print(f"T-statistic: {t_stat:.6f}, p-value: {p_val:.6e}")

T-statistic: 21.178254, p-value: 7.020576e-76


### Just in case you want to do the statistical test manually

In [22]:
# Rememmber that s^2 is the sample variance we calculated with np.var()
# means and SDs
mean_gm = 0.0088
mean_wm = -0.0019
sd_gm   = 0.012
sd_wm   = 0.012
n       = 2  # number of subjects

# standard error of difference
se_diff = np.sqrt(sd_gm**2/n + sd_wm**2/n)

# t-statistic
t = (mean_gm - mean_wm) / se_diff

# degrees of freedom (Welch-Satterthwaite)
df = (sd_gm**2/n + sd_wm**2/n)**2 / ((sd_gm**2/n)**2/(n-1) + (sd_wm**2/n)**2/(n-1))

# two-tailed p-value
p = 2 * (1 - stats.t.cdf(np.abs(t), df))

print("t =", t)
print("p =", p)


t = 0.8916666666666667
p = 0.4666574532668242


# <span style="color:#3BC6D9"> Chimap analysis of Spinal Cord Injury participants - swiss mk2 data // *For SC QSM* </br>

In [64]:
# Load the metrics created with SCT:
sci2_m1_gm_csv = pd.read_csv(r"E:\msc_data\sc_qsm\swiss_data_mk2\QSM_processing\sci2\m1\sci2_m1_gm_pipeline_mk1_metrics.csv")
sci2_m1_gm_metric_values = sci2_m1_gm_csv['WA()'][::-1]
sci2_m1_gm_metric_values


15   -0.000736
14    0.006754
13    0.009589
12    0.013587
11    0.006793
10    0.009126
9     0.012447
8     0.009862
7     0.006446
6     0.011258
5     0.004962
4     0.011965
3     0.016254
2     0.008775
1     0.001035
0     0.000852
Name: WA(), dtype: float64

In [65]:
sci2_m1_wm_csv = pd.read_csv(r"E:\msc_data\sc_qsm\swiss_data_mk2\QSM_processing\sci2\m1\sci2_m1_wm_pipeline_mk1_metrics.csv")
sci2_m1_wm_metric_values = sci2_m1_wm_csv['WA()'][::-1]
sci2_m1_wm_metric_values

15   -0.000504
14   -0.000745
13   -0.001522
12   -0.002189
11   -0.001970
10   -0.001687
9    -0.003241
8    -0.003181
7     0.000022
6    -0.003642
5    -0.002948
4    -0.000791
3    -0.004825
2    -0.002008
1    -0.000055
0    -0.001399
Name: WA(), dtype: float64

In [80]:
# Calculate mean and std
sci2_m1_gm_mean = np.mean(sci2_m1_gm_metric_values)
sci2_m1_wm_mean = np.mean(sci2_m1_wm_metric_values)

sci2_m1_gm_std = np.std(sci2_m1_gm_metric_values)
sci2_m1_wm_std = np.std(sci2_m1_wm_metric_values)

print(f"GM mean: {sci2_m1_gm_mean} ± {sci2_m1_gm_std} and WM mean: {sci2_m1_wm_mean} ± {sci2_m1_wm_std}")

GM mean: 0.008060578165 ± 0.0046355907636586955 and WM mean: -0.0019177784684999998 ± 0.0013292677024513803


# <span style="color:#D941C8"> Chimap analysis - chi_003 data // *For SC QSM* </br>

In [75]:
chi_003_pipeline_mk1_chimap_path = r"E:\msc_data\sc_qsm\neuropoly_data\chi_003\qsm_processing\2nd_3D_meGRE\auto_pipeline_running\mk1\chimap\Sepia_Chimap.nii.gz"
chi_003_gm_msk_path = r"E:\msc_data\sc_qsm\neuropoly_data\chi_003\qsm_processing\2nd_3D_meGRE\chi_003_m2_gm_msk.nii.gz"
chi_003_wm_msk_path = r"E:\msc_data\sc_qsm\neuropoly_data\chi_003\qsm_processing\2nd_3D_meGRE\chi_003_m2_wm_msk.nii.gz"

In [ ]:
# Load the metrics created with SCT:
chi003_m2_gm_csv = pd.read_csv(r"E:\msc_data\sc_qsm\neuropoly_data\chi_003\qsm_processing\2nd_3D_meGRE\chi_003_m2_gm_pipeline_mk1_metrics.csv")
chi003_m2_gm_metric_values = chi003_m2_gm_csv['WA()'][::-1]
chi003_m2_gm_metric_values

15    0.000000
14   -0.000355
13    0.004862
12    0.009333
11    0.010045
10    0.015117
9     0.019008
8     0.012465
7     0.018536
6     0.018662
5     0.016851
4     0.012294
3     0.009763
2     0.014367
1     0.008094
0     0.001150
Name: WA(), dtype: float64

In [72]:
# Crop from 13 to 3 and calculate statistics:
chi_003_m2_gm_mean = np.mean(chi003_m2_gm_metric_values[3:14])
print("Mean GM of chi_003 m2:", chi_003_m2_gm_mean)
chi_003_m2_gm_std =  np.std(chi003_m2_gm_metric_values[3:14])
print("STD GM of chi_003 m2:", chi_003_m2_gm_std)

Mean GM of chi_003 m2: 0.0142219475
STD GM of chi_003 m2: 0.0035290645032639706


In [67]:
# Load the metrics created with SCT:
chi003_m2_wm_csv = pd.read_csv(r"E:\msc_data\sc_qsm\neuropoly_data\chi_003\qsm_processing\2nd_3D_meGRE\chi_003_m2_wm_pipeline_mk1_metrics.csv")
chi003_m2_wm_metric_values = chi003_m2_wm_csv['WA()'][::-1]
chi003_m2_wm_metric_values

15         NaN
14   -0.000403
13   -0.001270
12   -0.001406
11   -0.001800
10   -0.002654
9    -0.002006
8    -0.004381
7    -0.003008
6    -0.002566
5    -0.003207
4    -0.001805
3    -0.003651
2    -0.003441
1    -0.001800
0     0.000352
Name: WA(), dtype: float64

In [73]:
# Crop from 13 to 3 and calculate statistics:
chi_003_m2_wm_mean = np.mean(chi003_m2_wm_metric_values[3:14])
print("Mean GM of chi_003 m2:", chi_003_m2_wm_mean)
chi_003_m2_wm_std =  np.std(chi003_m2_wm_metric_values[3:14])
print("STD GM of chi_003 m2:", chi_003_m2_wm_std)

Mean GM of chi_003 m2: -0.0027204642454545453
STD GM of chi_003 m2: 0.0008743824587737923


In [76]:
t_stat_chi_003_m2, p_val_chi_003_m2 = calculate_t_statistic_between_gm_wm(chi_003_pipeline_mk1_chimap_path, chi_003_wm_msk_path, chi_003_gm_msk_path)

T-statistic: 30.544083, p-value: 2.712096e-152
